# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [14]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
import json
from pprint import pprint
%matplotlib inline
from scipy.stats import linregress
from matplotlib import pyplot as plt
from api_keys import g_key 

# Import API key
import api_keys as api

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacation_path = 'city_weather.csv'
vacation_df = pd.read_csv(vacation_path)
vacation_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Punta Arenas,-53.1500,-70.9167,39.20,87,0,10.36,CL,1618453749
1,1,Fairbanks,64.8378,-147.7164,46.40,56,90,6.91,US,1618453479
2,2,Kapaa,22.0752,-159.3190,78.80,57,20,16.11,US,1618453749
3,3,Mataura,-46.1927,168.8643,62.01,73,38,5.99,NZ,1618453750
4,4,Coquimbo,-29.9533,-71.3436,59.00,88,90,1.14,CL,1618453463
...,...,...,...,...,...,...,...,...,...,...
533,533,Hualmay,-11.0964,-77.6139,66.56,85,86,7.09,PE,1618453972
534,534,Esmeraldas,-19.7625,-44.3139,68.00,68,0,11.50,BR,1618453840
535,535,Gayéri,12.6497,0.4931,86.11,26,56,4.83,BF,1618453972
536,536,Pindi Bhattian,31.8978,73.2730,82.72,29,93,10.13,PK,1618453972


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
gmaps.configure(api_key=g_key)
# Store 'Lat' and 'Lng' into  locations 
locations = vacation_df[["Lat", "Lng"]].astype(float)

# Convert humidity to float and store
# HINT: be sure to handle NaN values
#vacation_df = vacaton_df.dropna()
humidity_perc = vacation_df.Humidity.astype(float)

# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_perc, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
vacation_df["Cloudiness"].value_counts()

100    76
0      59
20     39
75     38
90     37
       ..
68      1
28      1
39      1
69      1
59      1
Name: Cloudiness, Length: 91, dtype: int64

In [5]:
max_temp = 85
min_temp = 75
ideal_vacay_df = vacation_df.loc[(vacation_df["Max Temp"] <= max_temp) & (vacation_df["Max Temp"] >= min_temp) &(vacation_df.Cloudiness <=1) & (vacation_df["Wind Speed"]<=10)]
ideal_vacay_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
73,73,Nelson Bay,-32.7167,152.1500,75.20,41,0,5.75,AU,1618453778
130,130,Lüderitz,-26.6481,15.1594,75.56,28,0,6.62,NaN,1618453796
209,209,Ixtapa,20.7000,-105.2000,77.00,78,1,5.75,MX,1618453537
268,268,Flinders,-34.5833,150.8552,75.99,42,0,4.00,AU,1618453853
273,273,Iranshahr,27.2025,60.6848,77.00,27,0,4.61,IR,1618453855
295,295,Kavaratti,10.5669,72.6420,83.89,67,1,1.03,IN,1618453867
355,355,Kiama,-34.6833,150.8667,75.99,42,1,4.00,AU,1618453895
360,360,Cabo San Lucas,22.8909,-109.9124,84.20,37,1,5.75,MX,1618453613
425,425,Araouane,18.9048,-3.5265,84.04,13,0,9.91,ML,1618453919
444,444,San Lorenzo,-25.3333,-57.5333,75.20,88,0,3.44,PY,1618453929


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
ideal_vacay_df.dropna(inplace=True)
ideal_vacay_df

<ipython-input-6-a2923cc032d4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_vacay_df.dropna(inplace=True)


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
73,73,Nelson Bay,-32.7167,152.1500,75.20,41,0,5.75,AU,1618453778
209,209,Ixtapa,20.7000,-105.2000,77.00,78,1,5.75,MX,1618453537
268,268,Flinders,-34.5833,150.8552,75.99,42,0,4.00,AU,1618453853
273,273,Iranshahr,27.2025,60.6848,77.00,27,0,4.61,IR,1618453855
295,295,Kavaratti,10.5669,72.6420,83.89,67,1,1.03,IN,1618453867
355,355,Kiama,-34.6833,150.8667,75.99,42,1,4.00,AU,1618453895
360,360,Cabo San Lucas,22.8909,-109.9124,84.20,37,1,5.75,MX,1618453613
425,425,Araouane,18.9048,-3.5265,84.04,13,0,9.91,ML,1618453919
444,444,San Lorenzo,-25.3333,-57.5333,75.20,88,0,3.44,PY,1618453929
514,514,Malakal,9.5334,31.6605,79.88,12,1,4.94,SS,1618453954


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [44]:
#hotel_name = []
#ideal_vacay_df.insert(9, "Hotel Name", True)
hotel_df = ideal_vacay_df[["City", "Country", "Lat", "Lng", "Humidity"]]
#hotel_df["Hotel Name"]=""
hotel_df

,City,Country,Lat,Lng,Humidity
73,Nelson Bay,AU,-32.7167,152.1500,41
209,Ixtapa,MX,20.7000,-105.2000,78
268,Flinders,AU,-34.5833,150.8552,42
273,Iranshahr,IR,27.2025,60.6848,27
295,Kavaratti,IN,10.5669,72.6420,67
355,Kiama,AU,-34.6833,150.8667,42
360,Cabo San Lucas,MX,22.8909,-109.9124,37
425,Araouane,ML,18.9048,-3.5265,13
444,San Lorenzo,PY,-25.3333,-57.5333,88
514,Malakal,SS,9.5334,31.6605,12


In [45]:
#cities = ideal_vacay_df.City
lat = hotel_df.Lat
lat
lng = hotel_df.Lng
#lng

In [47]:
# params dictionary to update each iteration
#hotel_name = []

params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotel",
    "key": api.g_key
}

# Use the lat/lng to identify hotels
for index, row in hotel_df.iterrows():
       # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotel_search = requests.get(base_url, params=params)
    
    # convert to json
    hotel_search = hotel_search.json()

    #print(json.dumps(hotel_search, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        #create and store hotel name and hotel rating
        hotel_df.loc[index, "Hotel Name"] = hotel_search["results"][0]["name"]
        hotel_df.loc[index, "Hotel Rating"] = hotel_search["results"][0]["rating"]
       
        #hotel_name.append(hotel_search["results"][0]["name"])
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        #hotel_name.append(np.nan(inplace=True))

hotel_df

C:\Users\bethg\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,City,Country,Lat,Lng,Humidity,Hotel Rating,Hotel Name
73,Nelson Bay,AU,-32.7167,152.1500,41,4.5,Amarna Luxury Beach Resort - Guest Only
209,Ixtapa,MX,20.7000,-105.2000,78,4.5,Hotel Mio Vallarta
268,Flinders,AU,-34.5833,150.8552,42,5.0,Ocean Farm
273,Iranshahr,IR,27.2025,60.6848,27,0.0,هتل خنونه سرزه
295,Kavaratti,IN,10.5669,72.6420,67,4.6,Bangaram Island Resort
355,Kiama,AU,-34.6833,150.8667,42,5.0,Ocean Farm
360,Cabo San Lucas,MX,22.8909,-109.9124,37,4.7,Nobu Hotel Los Cabos
425,Araouane,ML,18.9048,-3.5265,13,NaN,NaN
444,San Lorenzo,PY,-25.3333,-57.5333,88,4.7,Factoria Hotel
514,Malakal,SS,9.5334,31.6605,12,5.0,Nile Palace Hotel


In [48]:
hotel_df.to_csv("hotel.csv")

In [49]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]].astype(float)

In [52]:
gmaps.configure(api_key=g_key)
locations = hotel_df[["Lat", "Lng"]].astype(float)

humidity_perc = hotel_df.Humidity.astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_perc, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 5)

fig.add_layer(heat_layer)

#fig.add_layer(heat_layer)
markers = gmaps.marker_layer(locations)

fig.add_layer(heat_layer)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))